In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.models as models
import numpy as np
from thop import profile
from thop import clever_format
import time

In [2]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=7, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(2744, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3 # RGB : 3 // Y : 1
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [22, 21, 'M', 60, 57, 'M', 94, 97, 65, 'M', 80, 90, 75, 'M', 55, 76, 56, 'M'], ##
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def _vgg(arch, cfg, batch_norm, pretrained, progress, **kwargs):
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def vgg16(pretrained=False, progress=True, **kwargs):
    r"""VGG 16-layer model (configuration "D")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg16', 'D', False, pretrained, progress, **kwargs)


In [3]:
net_dir = 'pruning_result/VGG16_RGB_oneshot_80.pth'
net = torch.load(net_dir)

C:\Users\whtne\Anaconda3\envs\pythorch\lib\site-packages\torch\serialization.py:400: UserWarning: Couldn't retrieve source code for container of type VGG. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "
C:\Users\whtne\Anaconda3\envs\pythorch\lib\site-packages\torch\serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\whtne\Anaconda3\envs\pythorch\lib\site-packages\torch\serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to rever

In [4]:
# net = torch.load('C:/Users/whtne/Desktop/Dropbox/LAB/DIPL_18/2019-project-2-YYY/vs_RandomPruning/VGG16_RGB_onetshot70_prune.pth')
net = vgg16()
net.load_state_dict(torch.load(net_dir).state_dict())
net.dump_patches=True

In [5]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 22, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(22, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(21, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(60, 57, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(57, 94, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(94, 97, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(97, 65, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(65, 80, 

In [6]:
def computeTime(model, device='cuda'):
    inputs = torch.randn(1, 3, 224, 224) # input image 3ch for 3, 1ch for 1
    if device == 'cuda':
        model = model.cuda()
        inputs = inputs.cuda()

    model.eval()

    i = 0
    time_spent = []
    while i < 100:
        start_time = time.time()
        with torch.no_grad():
            out = model(inputs)

        if device == 'cuda':
            torch.cuda.synchronize()  # wait for cuda to finish (cuda is asynchronous!)
        if i != 0:
            time_spent.append(time.time() - start_time)
        i += 1
    print('Avg execution time: {:.3f}'.format(np.mean(time_spent)))
    print(np.mean(time_spent))

In [7]:
computeTime(net)

Avg execution time: 0.005
0.005228437558569089


In [8]:
net.cuda()
input = torch.randn(1, 3, 224, 224).cuda()
flops, params = profile(net, inputs=(input, ))
flops, params = clever_format([flops, params], "%.3f")

Register FLOP counter for module Conv2d(3, 22, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Register FLOP counter for module ReLU(inplace)
Register FLOP counter for module Conv2d(22, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Register FLOP counter for module ReLU(inplace)
Register FLOP counter for module MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Register FLOP counter for module Conv2d(21, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Register FLOP counter for module ReLU(inplace)
Register FLOP counter for module Conv2d(60, 57, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Register FLOP counter for module ReLU(inplace)
Register FLOP counter for module MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Register FLOP counter for module Conv2d(57, 94, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Register FLOP counter for module ReLU(inplace)
Register FLOP counter for module Conv2d(94, 97, kernel_

In [9]:
print(flops)

1.571G


In [10]:
sum([p.numel() for p in net.parameters() if p.requires_grad])

28573659